In [1]:
# imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import requests
import pickle
import io

In [2]:
# path to the dataset (replace with your own path) MAKE SURE YOU CHANGE THE SLASHES
path = "C:/Users/Asa/.cache/kagglehub/datasets/mostafaabla/garbage-classification/versions/1"

In [20]:
import preprocessing as prep
target_size = (150, 150)
X, y = prep.get_X_y(path=path, percent=0.5, target_size=target_size)

TypeError: get_X_y() got an unexpected keyword argument 'target_size'

In [4]:
y[:5]

,battery,biological,brown-glass,cardboard,clothes,green-glass,metal,paper,plastic,shoes,trash,white-glass
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
X.shape

(3720, 150, 150, 3)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [7]:
print("Train Lengths:", len(X_train), len(y_train))
print("Test Lengths:", len(X_test), len(y_test))

Train Lengths: 2790 2790
Test Lengths: 930 930


In [19]:
target_size + (3,)

NameError: name 'target_size' is not defined

In [ ]:
# defining the model
from tensorflow.keras import layers

input_shape = target_size + (3,)

model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(12, activation='sigmoid')  # 20 classes
])

c:\Users\Asa\miniconda3\envs\dev\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [16]:
batch_size = 32
epochs = 10
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs
)

Epoch 1/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 37s 386ms/step - accuracy: 0.1864 - loss: 2.3703 - val_accuracy: 0.4237 - val_loss: 1.6696
Epoch 2/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 34s 382ms/step - accuracy: 0.4644 - loss: 1.6452 - val_accuracy: 0.4903 - val_loss: 1.4851
Epoch 3/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 32s 369ms/step - accuracy: 0.5679 - loss: 1.2783 - val_accuracy: 0.5161 - val_loss: 1.4709
Epoch 4/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 32s 368ms/step - accuracy: 0.6804 - loss: 0.9661 - val_accuracy: 0.4710 - val_loss: 1.6216
Epoch 5/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 32s 361ms/step - accuracy: 0.7758 - loss: 0.7169 - val_accuracy: 0.5183 - val_loss: 1.7826
Epoch 6/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 31s 351ms/step - accuracy: 0.8368 - loss: 0.5034 - val_accuracy: 0.4935 - val_loss: 2.2198
Epoch 7/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 33s 371ms/step - accuracy: 0.9136 - loss: 0.2680 - val_accuracy: 0.4935 - val_loss: 2.4656
Epoch 8/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 31s 356ms/step - accuracy: 0.9374 - loss: 0.2116 - val_accu

In [17]:
# Evaluate the model using the testing data
model.evaluate(X_test, y_test)

30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - accuracy: 0.5081 - loss: 3.2722


[3.372894763946533, 0.4956989288330078]

In [18]:
# Saving the model
with open("initial_model.pkl", "wb") as f:
    pickle.dump(model, f)